In [10]:
import os
import random
import time
import wandb
from distutils.util import strtobool
from typing import Callable
import flax
import flax.linen as nn
import gymnasium as gym
import jax
import jax.numpy as jnp
import numpy as np
import optax

from flax.training.train_state import TrainState
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter

In [11]:
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env, f"video/{run_name}")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)

        return env

    return thunk

In [12]:
class QNetwork(nn.Module):
    action_dim: int

    @nn.compact
    def __call__(self, x: jnp.ndarray):
        x = nn.Dense(120)(x)
        x = nn.relu(x)
        x = nn.Dense(84)(x)
        x = nn.relu(x)
        x = nn.Dense(self.action_dim)(x)
        # x = nn.relu(x)

        return x


In [13]:
class TrainState(TrainState):
    target_params: flax.core.FrozenDict

In [14]:
def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e)/ duration
    return max(slope * t + start_e, end_e)

In [15]:
def evaluate(model_path: str,
             make_env: Callable,
             env_id: str,
             eval_episodes: int,
             run_name: str,
             Model: nn.Module,
             epsilon: float = 0.05,
             capture_video: bool = True,
             seed=1):
    envs = gym.vector.SyncVectorEnv([make_env(env_id, 0, 0, capture_video, run_name)])
    obs, _ = envs.reset()
    model = Model(action_dim=envs.single_action_space.n)
    q_key = jax.random.PRNGKey(seed)
    params = model.init(q_key, obs)
    with open(model_path, "rb") as f:
        params = flax.serialization.from_bytes(params, f.read())
    model.apply = jax.jit(model.apply)

    episodic_returns = []

    while len(episodic_returns) < eval_episodes:
        if random.random() < epsilon:
            actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
        else:
            q_values = model.apply(params,obs)
            actions = q_values.argmax(axis=-1)
            actions = jax.device_get(actions)
        next_obs, _, _, _, infos = envs.step(actions)
        if "final_info" in infos:
            for info in infos["final_info"]:
                if "episode" not in info:
                    continue
                print(f"eval_episode={len(episodic_returns)}, episodic_return={info['episode']['r']}")
                episodic_returns += [info["episode"]["r"]]

        obs = next_obs

    return episodic_returns

In [16]:
envs = {"cartPole":"CartPole-v1","acrobot":"Acrobot-v1","mountainCar":"MountainCar-v0"}
n_timesteps = 500000
learning_rate = 0.00025
buffer_size = 10000
gamma = 0.99
target_network_frequency = 500
max_grad_norm = 0.5
batch_size = 1024
start_e = 1
end_e = 0.05
exploration_fraction = 0.5
learning_starts = 10000
train_frequency = 10
seed = 1
capture_video = True
tau = 1
num_envs = 1

In [17]:
def train(env_id,timesteps,learning_rate,buffer_size,gamma,
          target_network_frequency,max_grad_norm,batch_size,
          start_e,end_e,exploration_fraction,learning_starts,
          train_frequency,seed,tau,num_envs):

    run_name = f"{env_id}__{seed}__{int(time.time())}"
    wandb.init(
        project="rl-algos-benchmark",
        config={
            "env":env_id,
            "timesteps":timesteps,
            "lr":learning_rate,
            "buffer_size":buffer_size,
            "gamma":gamma,
            "target_network_frequency":target_network_frequency,
            "batch_size":batch_size,
            "start_e":start_e,
            "end_e":end_e,
            "exploration_fraction":exploration_fraction,
            "learning_starts":learning_starts,
            "train_frequency":train_frequency,
            "seed":seed,
            "tau":tau,
            "num_envs":num_envs,
        },
        sync_tensorboard=True,
        monitor_gym=True,
        name=run_name,
    )
    
    writer = SummaryWriter(f"runs/{run_name}")

    random.seed(seed)
    np.random.seed(seed)
    key = jax.random.PRNGKey(seed)
    key, q_key = jax.random.split(key, 2)

    envs = gym.vector.SyncVectorEnv(
        [make_env(env_id, seed + i, i, True, run_name) for i in range(num_envs)]
    )

    obs, _ = envs.reset(seed=seed)

    q_network = QNetwork(action_dim=envs.single_action_space.n)

    q_state = TrainState.create(
        apply_fn=q_network.apply,
        params=q_network.init(q_key, obs),
        target_params=q_network.init(q_key,obs),
        tx=optax.adam(learning_rate=learning_rate)
    )

    q_network.apply = jax.jit(q_network.apply)
    q_state = q_state.replace(target_params=optax.incremental_update(q_state.params, q_state.target_params, 1))

    rb = ReplayBuffer(
        buffer_size,
        envs.single_observation_space,
        envs.single_action_space,
        "cpu",
        n_envs=num_envs,
        handle_timeout_termination=False,
    )

    @jax.jit
    def update(q_state, observations, actions, next_observations, rewards, dones):
        q_next_target = q_network.apply(q_state.target_params, next_observations)
        q_next_target = jnp.max(q_next_target, axis=-1)
        next_q_value = rewards + (1 - dones) * gamma * q_next_target

        def mse_loss(params):
            q_pred = q_network.apply(params, observations)
            q_pred = q_pred[jnp.arange(q_pred.shape[0]), actions.squeeze()]
            return ((q_pred - next_q_value)**2).mean(), q_pred

        (loss_value,q_pred), grads = jax.value_and_grad(mse_loss,has_aux=True)(q_state.params)
        q_state = q_state.apply_gradients(grads=grads)
        return loss_value, q_pred, q_state

    start_time = time.time()

    obs, _ = envs.reset(seed=seed)
    for global_step in range(timesteps):
        epsilon = linear_schedule(start_e, end_e, exploration_fraction * timesteps, global_step)
        if random.random() < epsilon:
            actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
        else:
            q_values = q_network.apply(q_state.params, obs)
            actions = q_values.argmax(axis=-1)
            actions = jax.device_get(actions)

        next_obs, rewards, terminated, truncated, infos = envs.step(actions)

        if "final_info" in infos:
            for info in infos["final_info"]:
                if "episode" not in info:
                    continue

                print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
                writer.add_scalar("charts/episodic_return",info["episode"]["r"], global_step)
                writer.add_scalar("charts/episodic_length",info["episode"]["l"], global_step)
                writer.add_scalar("charts/epsilon",epsilon,global_step)


        real_next_obs = next_obs.copy()
        for idx,d in enumerate(truncated):
            if d and "final_observations" in infos:
                real_next_obs[idx] = infos["final_observations"][idx]
        rb.add(obs, real_next_obs, actions, rewards, terminated, infos)

        obs = next_obs

        if global_step > learning_starts:
            if global_step % train_frequency == 0:
                data = rb.sample(batch_size)

                loss, old_val, q_state = update(
                    q_state,
                    data.observations.numpy(),
                    data.actions.numpy(),
                    data.next_observations.numpy(),
                    data.rewards.flatten().numpy(),
                    data.dones.flatten().numpy(),
                )

                if global_step % 100 == 0:
                    writer.add_scalar("losses/td_loss",jax.device_get(loss),global_step)
                    writer.add_scalar("losses/q_values",jax.device_get(old_val).mean(), global_step)
                    writer.add_scalar("charts/SPS",int(global_step / (time.time() - start_time)),global_step)

            if global_step % target_network_frequency == 0:
                q_state = q_state.replace(
                    target_params=optax.incremental_update(q_state.params, q_state.target_params, tau)
                )

    model_path = f"runs/{run_name}.cleanrl_model"
    with open(model_path,"wb") as f:
        f.write(flax.serialization.to_bytes(q_state.params))

    print("Model Saved")

    episodic_returns = evaluate(
        model_path,
        make_env,
        env_id,
        run_name=f"{run_name}-eval",
        Model=QNetwork,
        epsilon=0.05,
        eval_episodes=100
    )

    for idx, episodic_return in enumerate(episodic_returns):
        writer.add_scalar("eval/episodic_return", episodic_return,idx)

    envs.close()
    writer.close()
    wandb.finish()

In [18]:
train(envs["acrobot"],n_timesteps,learning_rate,buffer_size,gamma,target_network_frequency,max_grad_norm,batch_size,start_e,end_e,exploration_fraction,learning_starts,train_frequency,seed,tau,num_envs)

wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in runs/Acrobot-v1__1__1699875373


Moviepy - Building video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-0.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-0.mp4
global_step=499, episodic_return=[-500.]
Moviepy - Building video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-1.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-1.mp4
global_step=999, episodic_return=[-500.]
global_step=1499, episodic_return=[-500.]
global_step=1999, episodic_return=[-500.]
global_step=2499, episodic_return=[-500.]
global_step=2999, episodic_return=[-500.]
global_step=3499, episodic_return=[-500.]
global_step=3999, episodic_return=[-500.]
Moviepy - Building video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-video-episode-8.mp4
global_step=4499, episodic_return=[-500.]
global_step=4999, episodic_return=[-500.]
global_step=5499, episodic_return=[-500.]
global_step=5999, episodic_return=[-500.]
global_step=6499, episodic_return=[-500.]
global_step=6999, episodic_return=[-500.]
global_step=7499, episodic_return=[-500.]
global_step=7999, episodic_return=[-500.]
global_step=8499, episodic_return=[-500.]
global_step=8999, episodic_return=[-500.]
global_step=9499, episodic_return=[-500.]
global_step=9999, episodic_return=[-500.]
global_step=10499, episodic_return=[-500.]
global_step=10999, episodic_return=[-500.]
global_step=11499, episodic_return=[-500.]
global_step=11999, episodic_return=[-500.]
global_step=12499, episodic_return=[-500.]
global_step=12999, episodic_return=[-500.]
global_step=13499, episodic_return=[-500.]
Moviepy - Building video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373/rl-v

...

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/Acrobot-v1__1__1699875373-eval/rl-video-episode-1.mp4
eval_episode=1, episodic_return=[-101.]
eval_episode=2, episodic_return=[-76.]
eval_episode=3, episodic_return=[-81.]
eval_episode=4, episodic_return=[-99.]
eval_episode=5, episodic_return=[-75.]
eval_episode=6, episodic_return=[-90.]
eval_episode=7, episodic_return=[-69.]
Moviepy - Building video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373-eval/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373-eval/rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/Acrobot-v1__1__1699875373-eval/rl-video-episode-8.mp4
eval_episode=8, episodic_return=[-99.]
eval_episode=9, episodic_return=[-82.]
eval_episode=10, episodic_return=[-86.]
eval_episode=11, episodic_return=[-63.]
eval_episode=12, episodic_return=[-96.]
eval_episode=13, episodic_return=[-92.]
eval_episode=14, episodic_return=[-63.]
eval_episode=15, episodic_return=[-78.]
eval_episode=16, episodic_return=[-77.]
eval_episode=17, episodic_return=[-79.]
eval_episode=18, episodic_return=[-86.]
eval_episode=19, episodic_return=[-92.]
eval_episode=20, episodic_return=[-69.]
eval_episode=21, episodic_return=[-80.]
eval_episode=22, episodic_return=[-84.]
eval_episode=23, episodic_return=[-63.]
eval_episode=24, episodic_return=[-115.]
eval_episode=25, episodic_return=[-68.]
eval_episode=26, episodic_return=[-69.]
Moviepy - Building video /notebooks/rl-algos/video/Acrobot-v1__1__1699875373-eval/rl-video-episode-27.mp4.
Moviepy - Writi

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/Acrobot-v1__1__1699875373-eval/rl-video-episode-27.mp4
eval_episode=27, episodic_return=[-74.]
eval_episode=28, episodic_return=[-80.]
eval_episode=29, episodic_return=[-83.]
eval_episode=30, episodic_return=[-82.]
eval_episode=31, episodic_return=[-76.]
eval_episode=32, episodic_return=[-103.]
eval_episode=33, episodic_return=[-70.]
eval_episode=34, episodic_return=[-75.]
eval_episode=35, episodic_return=[-91.]
eval_episode=36, episodic_return=[-83.]
eval_episode=37, episodic_return=[-78.]
eval_episode=38, episodic_return=[-108.]
eval_episode=39, episodic_return=[-76.]
eval_episode=40, episodic_return=[-100.]
eval_episode=41, episodic_return=[-126.]
eval_episode=42, episodic_return=[-85.]
eval_episode=43, episodic_return=[-102.]
eval_episode=44, episodic_return=[-92.]
eval_episode=45, episodic_return=[-85.]
eval_episode=46, episodic_return=[-143.]
eval_episode=47, episodic_return=[-89.]
eval_episode=48, episodic_return=[

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/Acrobot-v1__1__1699875373-eval/rl-video-episode-64.mp4
eval_episode=64, episodic_return=[-82.]
eval_episode=65, episodic_return=[-79.]
eval_episode=66, episodic_return=[-86.]
eval_episode=67, episodic_return=[-87.]
eval_episode=68, episodic_return=[-94.]
eval_episode=69, episodic_return=[-87.]
eval_episode=70, episodic_return=[-82.]
eval_episode=71, episodic_return=[-75.]
eval_episode=72, episodic_return=[-64.]
eval_episode=73, episodic_return=[-75.]
eval_episode=74, episodic_return=[-74.]
eval_episode=75, episodic_return=[-71.]
eval_episode=76, episodic_return=[-118.]
eval_episode=77, episodic_return=[-75.]
eval_episode=78, episodic_return=[-90.]
eval_episode=79, episodic_return=[-69.]
eval_episode=80, episodic_return=[-69.]
eval_episode=81, episodic_return=[-101.]
eval_episode=82, episodic_return=[-95.]
eval_episode=83, episodic_return=[-72.]
eval_episode=84, episodic_return=[-69.]
eval_episode=85, episodic_return=[-121

charts/SPS,▁▃▅▇▇████████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅
charts/episodic_length,█▅▄▄▂▃▂▂▂▃▂▂▂▁▃▂▁▂▁▂▁▂▁▁▁▁▁▁▁▂▁▂▁▃▁▂▁▁▂▂
charts/episodic_return,▁▄▅▅▇▆▇▇▇▆▇▇▇█▆▇█▇█▇█▇███████▇█▇█▆█▇██▇▇
charts/epsilon,█▇▆▅▅▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/episodic_return,▇▇▇█▅▄▇▆█▆█▇▆█▅▂▄▃▆▄▆▆▃▅▃▅▅▄▇▇▁▅▃▇█▅▄▆▄▅
global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
losses/q_values,█▆▃▁▁▂▃▄▄▄▃▄▃▄▃▄▃▄▄▃▄▃▃▄▃▄▄▄▃▄▄▄▃▄▃▄▄▄▄▄
losses/td_loss,▁▂▂▅█▅▄▃▂▃▂▃▃▃▃▃▃▃▃▂▃▃▃▄▃▄▄▄▃▃▃▃▃▄▄▄▄▃▃▃
charts/SPS,1173.0
charts/episodic_length,64.0
charts/episodic_return,-63.0
